# Análisis de Datos - Machine Learning

Este notebook contiene el análisis completo de datos con modelos de machine learning.

## Pasos del análisis:
1. Carga de datos
2. Conversión a JSON y conteo por cuenta
3. Análisis Exploratorio de Datos (EDA)
4. Filtrado de datos (Tarifa NBO y Rentabilizacion)
5. Preparación de datos
6. Entrenamiento de modelos (Regresión Logística, Random Forest, XGBoost)
7. Validación cruzada
8. Grid Search para árboles de decisión
9. Comparación y selección del mejor modelo


## Paso 1: Importación de librerías


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
import warnings
import json
import pickle
from datetime import datetime
import os

warnings.filterwarnings('ignore')

print("✓ Librerías importadas correctamente")


## Paso 2: Carga de datos


In [ ]:
archivo = "Total_Mes_Act_Datos completos.csv"
output_dir = 'resultados'
os.makedirs(output_dir, exist_ok=True)

print("Cargando datos...")
try:
    df = pd.read_csv(
        archivo, 
        low_memory=False,
        encoding='utf-8',
        on_bad_lines='skip',
        sep=',',
        quotechar='"'
    )
    print(f"✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
except Exception as e:
    print(f"Error: {e}")
    print("Intentando con encoding alternativo...")
    df = pd.read_csv(
        archivo,
        low_memory=False,
        encoding='latin-1',
        on_bad_lines='skip',
        sep=';',
        quotechar='"'
    )
    print(f"✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")

df.head()


## Paso 3: Conversión a JSON y Conteo por Cuenta


In [ ]:
print("Convirtiendo datos a JSON (muestra de 100,000 registros)...")
max_rows_json = min(100000, len(df))
df_muestra = df.head(max_rows_json)

ruta_json = os.path.join(output_dir, 'datos_completos.json')
df_muestra.to_json(
    ruta_json,
    orient='records',
    date_format='iso',
    indent=2,
    force_ascii=False
)
print(f"✓ Archivo JSON guardado en '{ruta_json}'")
print(f"Total de registros convertidos: {len(df_muestra)}")


In [ ]:
columna_cuenta = None

for col in df.columns:
    if 'cuenta' in col.lower():
        columna_cuenta = col
        break

if columna_cuenta is None:
    posibles = [col for col in df.columns if 'cuent' in col.lower() or 'account' in col.lower()]
    if posibles:
        columna_cuenta = posibles[0]

if columna_cuenta:
    print(f"Columna seleccionada: {columna_cuenta}")
    conteo = df[columna_cuenta].value_counts().reset_index()
    conteo.columns = ['Cuenta', 'Frecuencia']
    conteo = conteo.sort_values('Frecuencia', ascending=False)
    
    print(f"\nTotal de cuentas únicas: {len(conteo)}")
    print(f"\nTop 10 cuentas más frecuentes:")
    display(conteo.head(10))
    
    archivo_csv = os.path.join(output_dir, 'conteo_por_cuenta.csv')
    conteo.to_csv(archivo_csv, index=False, encoding='utf-8')
    print(f"\n✓ Conteo guardado en '{archivo_csv}'")
else:
    print("No se encontró columna 'cuenta'")


## Paso 4: Análisis Exploratorio de Datos (EDA)


In [ ]:
print("Información del dataset:")
print(f"Dimensiones: {df.shape}")
print(f"\nColumnas: {len(df.columns)}")
print(f"\nTipos de datos:")
print(df.dtypes.value_counts())

print(f"\nValores faltantes:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Valores_Faltantes': missing,
    'Porcentaje': missing_pct
})
missing_df = missing_df[missing_df['Valores_Faltantes'] > 0].sort_values('Porcentaje', ascending=False)

if len(missing_df) > 0:
    display(missing_df.head(20))
else:
    print("No hay valores faltantes")


## Paso 5: Identificación y Filtrado de Datos


In [ ]:
columna_tarifa = None
columna_target = None

for col in df.columns:
    if 'tarifa' in col.lower() and 'nbo' in col.lower():
        columna_tarifa = col
        break

if columna_tarifa is None:
    posibles = [col for col in df.columns if 'tarifa' in col.lower() or 'nbo' in col.lower()]
    if posibles:
        columna_tarifa = posibles[0]

for col in df.columns:
    if 'rentabilizacion' in col.lower() or 'rentabiliz' in col.lower():
        columna_target = col
        break

if columna_target is None:
    posibles = [col for col in df.columns if 'rent' in col.lower()]
    if posibles:
        columna_target = posibles[0]

print(f"Columna Tarifa NBO: {columna_tarifa}")
print(f"Columna Rentabilizacion: {columna_target}")

if columna_tarifa and columna_target:
    df_filtrado = df[df[columna_tarifa].notna()].copy()
    print(f"\nFilas después de filtrar por {columna_tarifa}: {len(df_filtrado)}")
    
    df_filtrado = df_filtrado[df_filtrado[columna_target].notna()].copy()
    print(f"Filas después de filtrar por {columna_target}: {len(df_filtrado)}")
    
    print(f"\nDistribución de la variable objetivo ({columna_target}):")
    display(df_filtrado[columna_target].value_counts())
else:
    print("\nError: No se encontraron las columnas necesarias")
    df_filtrado = None


## Paso 6: Preparación de Datos para Modelado


In [ ]:
if df_filtrado is not None and columna_target:
    y = df_filtrado[columna_target].copy()
    X = df_filtrado.drop(columns=[columna_target])
    
    columnas_numericas = X.select_dtypes(include=[np.number]).columns.tolist()
    columnas_categoricas = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"Columnas numéricas: {len(columnas_numericas)}")
    print(f"Columnas categóricas: {len(columnas_categoricas)}")
    
    X_processed = X[columnas_numericas].copy()
    
    for col in columnas_categoricas:
        if X[col].nunique() < 50:
            le = LabelEncoder()
            X_processed[col] = le.fit_transform(X[col].astype(str).fillna('Missing'))
    
    X_processed = X_processed.fillna(X_processed.median())
    
    if isinstance(y.dtype, object) or y.dtype == 'object':
        le_target = LabelEncoder()
        y = le_target.fit_transform(y.astype(str))
    else:
        y = y.astype(int)
    
    print(f"\nShape final: X={X_processed.shape}, y={y.shape}")
    print(f"Clases en y: {np.unique(y)}")
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\n✓ Datos preparados:")
    print(f"  Train: {X_train.shape[0]} muestras")
    print(f"  Test: {X_test.shape[0]} muestras")
else:
    print("Error: No se pueden preparar los datos")


## Paso 7: Entrenamiento de Modelos

Dividimos el flujo en subpasos utilizando lecturas por bloques de 50.000 filas para monitorear el progreso y reutilizar los artefactos generados.


## Paso 8: Grid Search con muestreo estratificado


In [ ]:
if 'X_processed' not in globals() or 'y' not in globals():
    raise RuntimeError('Es necesario ejecutar el Paso 7.1.1 antes de continuar.')

n_muestra = min(300000, len(X_processed))
print(f"Tomando muestra estratificada de {n_muestra} registros para grid search...", flush=True)

X_muestra, _, y_muestra, _ = train_test_split(
    X_processed,
    y,
    train_size=n_muestra,
    random_state=42,
    stratify=y
)

X_train_grid, X_test_grid, y_train_grid, y_test_grid = train_test_split(
    X_muestra,
    y_muestra,
    test_size=0.2,
    random_state=42,
    stratify=y_muestra
)

print(f"  Train: {X_train_grid.shape[0]} muestras", flush=True)
print(f"  Test: {X_test_grid.shape[0]} muestras", flush=True)

cv_grid = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

print('\nGrid Search - Random Forest', flush=True)
parametros_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)
modelo_grid_rf = GridSearchCV(
    rf_base,
    parametros_rf,
    cv=cv_grid,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)
modelo_grid_rf.fit(X_train_grid, y_train_grid)

resultado_grid_rf = {
    'best_params': modelo_grid_rf.best_params_,
    'best_score': float(modelo_grid_rf.best_score_)
}

with open(os.path.join(output_dir, 'grid_rf.json'), 'w', encoding='utf-8') as archivo_rf:
    json.dump(resultado_grid_rf, archivo_rf, indent=2)

print(f"✓ Mejor combinación RF: {resultado_grid_rf['best_params']} (score {resultado_grid_rf['best_score']:.4f})", flush=True)

print('\nGrid Search - XGBoost', flush=True)
parametros_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8, 1.0]
}

xgb_base = XGBClassifier(random_state=42, n_jobs=-1, eval_metric='logloss')
modelo_grid_xgb = GridSearchCV(
    xgb_base,
    parametros_xgb,
    cv=cv_grid,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)
modelo_grid_xgb.fit(X_train_grid, y_train_grid)

resultado_grid_xgb = {
    'best_params': modelo_grid_xgb.best_params_,
    'best_score': float(modelo_grid_xgb.best_score_)
}

with open(os.path.join(output_dir, 'grid_xgb.json'), 'w', encoding='utf-8') as archivo_xgb:
    json.dump(resultado_grid_xgb, archivo_xgb, indent=2)

print(f"✓ Mejor combinación XGBoost: {resultado_grid_xgb['best_params']} (score {resultado_grid_xgb['best_score']:.4f})", flush=True)
print('\n✓ Paso 8 completado', flush=True)


## Paso 9: Resumen de resultados de Grid Search


In [ ]:
ruta_grid_rf = os.path.join(output_dir, 'grid_rf.json')
ruta_grid_xgb = os.path.join(output_dir, 'grid_xgb.json')

with open(ruta_grid_rf, 'r', encoding='utf-8') as archivo_rf:
    resumen_rf = json.load(archivo_rf)

with open(ruta_grid_xgb, 'r', encoding='utf-8') as archivo_xgb:
    resumen_xgb = json.load(archivo_xgb)

print('Resultados guardados de Grid Search:', flush=True)
print(f"  Random Forest -> score CV: {resumen_rf['best_score']:.4f} | parámetros: {resumen_rf['best_params']}", flush=True)
print(f"  XGBoost       -> score CV: {resumen_xgb['best_score']:.4f} | parámetros: {resumen_xgb['best_params']}", flush=True)

if 'modelo_grid_rf' in globals() and 'modelo_grid_xgb' in globals():
    print('\nEvaluación rápida en el conjunto de prueba retenido de la muestra:', flush=True)
    pred_rf = modelo_grid_rf.best_estimator_.predict(X_test_grid)
    pred_xgb = modelo_grid_xgb.best_estimator_.predict(X_test_grid)
    print(f"  Random Forest -> accuracy test: {accuracy_score(y_test_grid, pred_rf):.4f}", flush=True)
    print(f"  XGBoost       -> accuracy test: {accuracy_score(y_test_grid, pred_xgb):.4f}", flush=True)
else:
    print('\nPara evaluar en test es necesario volver a ejecutar el Paso 8 en esta sesión.', flush=True)


## Paso 10: Comparación y Conclusión


In [ ]:
ruta_grid_rf = os.path.join(output_dir, 'grid_rf.json')
ruta_grid_xgb = os.path.join(output_dir, 'grid_xgb.json')
ruta_cv_lr = os.path.join(output_dir, 'resultado_cv_lr.json')

faltantes = [ruta for ruta in [ruta_grid_rf, ruta_grid_xgb, ruta_cv_lr] if not os.path.exists(ruta)]
if faltantes:
    raise FileNotFoundError(f"No se encontraron los archivos requeridos: {faltantes}")

with open(ruta_grid_rf, 'r', encoding='utf-8') as archivo_rf:
    resumen_rf = json.load(archivo_rf)

with open(ruta_grid_xgb, 'r', encoding='utf-8') as archivo_xgb:
    resumen_xgb = json.load(archivo_xgb)

with open(ruta_cv_lr, 'r', encoding='utf-8') as archivo_lr:
    resumen_lr = json.load(archivo_lr)

print('COMPARACIÓN DE MODELOS', flush=True)
print('=' * 80, flush=True)
print('\nValidación cruzada - Regresión logística:', flush=True)
print(f"  Accuracy promedio: {resumen_lr['mean']:.4f} (+/- {resumen_lr['std'] * 2:.4f})", flush=True)

print('\nGrid Search - Modelos de árboles:', flush=True)
print(f"  Random Forest -> score CV: {resumen_rf['best_score']:.4f} | parámetros: {resumen_rf['best_params']}", flush=True)
print(f"  XGBoost       -> score CV: {resumen_xgb['best_score']:.4f} | parámetros: {resumen_xgb['best_params']}", flush=True)

mejor_arbol = 'RandomForest' if resumen_rf['best_score'] > resumen_xgb['best_score'] else 'XGBoost'

print('\n' + '=' * 80, flush=True)
print('CONCLUSIÓN', flush=True)
print('=' * 80, flush=True)
print(f"  Mejor modelo entre árboles (según Grid Search): {mejor_arbol}", flush=True)

conclusion = {
    'cv_lr_mean': resumen_lr['mean'],
    'cv_lr_std': resumen_lr['std'],
    'grid_rf': resumen_rf,
    'grid_xgb': resumen_xgb,
    'best_tree_model': mejor_arbol
}

ruta_conclusion = os.path.join(output_dir, 'conclusion_paso_10.json')
with open(ruta_conclusion, 'w', encoding='utf-8') as archivo_conclusion:
    json.dump(conclusion, archivo_conclusion, indent=2)

print(f"\n✓ Conclusión almacenada en: {ruta_conclusion}", flush=True)
